In [1]:
import httpx
import pandas as pd
import asyncio
import logging
from typing import Dict, Any
from sharpshooter.utils.utils import fetch_external_data, dict_to_dataframe
from sharpshooter.notebooks._models import DKDraftablesResponse

In [2]:


async def get_draftkings_draftables() -> Dict[str, Any]:
    """
    Fetch draftable players data from the DraftKings API.

    Returns:
        Dict[str, Any]: The JSON response containing draftable players data.

    Raises:
        httpx.RequestError: If there's a network-related error.
        httpx.HTTPStatusError: If the HTTP response is an error status code.
    """
    url = "https://api.draftkings.com/draftgroups/v1/draftgroups/112297/draftables?format=json"
    return await fetch_external_data(url)

In [2]:
async def fetch_draftkings_draftables(draft_group_id: int) -> pd.DataFrame:
    url = f"https://api.draftkings.com/draftgroups/v1/draftgroups/{draft_group_id}/draftables?format=json"
    data = await fetch_external_data(url)
    response = DKDraftablesResponse.model_validate(data)
    logging.info(
        f"Fetched {len(response.draftables)} draftables for draft group {draft_group_id}"
    )
    
    # Convert the draftables to a list of dictionaries
    draftables_list = [draftable.model_dump() for draftable in response.draftables]
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(draftables_list)
    
    return df

In [3]:
df = await fetch_draftkings_draftables(112297)

In [5]:
# Configure Pandas to Show ALl Columns and Rows
pd.set_option("display.max_columns", None)

In [8]:
def flatten_structs(df):
    def flatten_row(row):
        flat_data = {}
        
        # Flatten 'competition' dictionary
        competition = row['competition']
        flat_data['competitionId'] = competition['competitionId']
        flat_data['competition_name'] = competition['name']
        
        # Flatten 'competitions' list of dictionaries
        for i, comp in enumerate(row['competitions']):
            flat_data[f'competition_{i}_id'] = comp['competitionId']
            flat_data[f'competition_{i}_name'] = comp['name']
        
        # Flatten 'draftStatAttributes' list of dictionaries
        for i, stat in enumerate(row['draftStatAttributes']):
            flat_data[f'draftStat_{i}_id'] = stat['id']
            flat_data[f'draftStat_{i}_value'] = stat['value']
            flat_data[f'draftStat_{i}_sortValue'] = stat['sortValue']
        
        return flat_data
    
    # Apply the flatten_row function to each row of the DataFrame
    flattened_rows = df.apply(flatten_row, axis=1)
    
    # Convert the flattened rows into a new DataFrame
    flat_df = pd.DataFrame(flattened_rows.tolist())
    
    return flat_df

In [9]:
df_i = flatten_structs(df)

In [10]:
df_i.head()

,competitionId,competition_name,competition_0_id,competition_0_name,draftStat_0_id,draftStat_0_value,draftStat_0_sortValue,draftStat_1_id,draftStat_1_value,draftStat_1_sortValue
0,6041664,PHI @ CLE,6041664,PHI @ CLE,90,22.8,22.8,-2,-,
1,6041663,DEN @ TB,6041663,DEN @ TB,90,20.2,20.2,-2,-,
2,6041663,DEN @ TB,6041663,DEN @ TB,90,16.5,16.5,-2,-,
3,6041663,DEN @ TB,6041663,DEN @ TB,90,16.5,16.5,-2,-,
4,6041662,MIN @ DET,6041662,MIN @ DET,90,16.8,16.8,-2,-,
